In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 1. Basic Visualization; preprocessing the data - simple analysis such as counting, filtering, cleaning the data, and some basic plotting techniques

In [ ]:
# import standard libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import os
print(os.listdir("../input/pokemon"))

In [ ]:
pokemon_df = pd.read_csv("../input/pokemon/pokemon.csv")
## from the columns, pick out the ones of type float64, int64, and objects
df2 = pokemon_df.select_dtypes(include = ['float64', 'int64'])
df3 = pokemon_df.select_dtypes(include = ['object'])

In [ ]:
pokemon_df.info()

In [ ]:
df2.info()

In [ ]:
## .head() grabs the first 5 elements from the dataframe
df3.head()

In [ ]:
## .tail() grabs the last 5 elements from the dataFrame
df3.tail()

In [ ]:
## https://scikit-learn.org/stable/modules/preprocessing.html
## https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

## changes raw feature vectors into a representation 
## that is more suitable for the downstream estimators
from sklearn import preprocessing 

## from preprocessing, use LabelEncoder, we want to encode 
## labels with value between 0 and n_classes - 1
le = preprocessing.LabelEncoder() 
labels = le.fit_transform(pokemon_df['type1'])
print(len(le.classes_)) ## len() -> length, classes is number of classes
print(le.classes_) ## prints what those classes are from the labels

### I encourage you to play around with the data and come up with possible questions you might want to ask, here are a few

In [ ]:
## from the dataframe, based on types, plot the occurences 
pokemon_df.type1.value_counts().plot.bar()

## What if we wanted to know a specific value?
pokemon_df.type1.value_counts() 

In [ ]:
## As you noticed flying type Pokemon seem low, why?
## Recall there are type1 and type2, let's look at type 2
pokemon_df.type2.value_counts()
pokemon_df.type2.value_counts().plot.bar()

In [ ]:
## Now there's a lot more flying types, so let's combine them
pokemon_df.type1.value_counts() + pokemon_df.type2.value_counts()

## and the accurate plot, keep in mind not every pokemon has 2 types
(pokemon_df.type1.value_counts() + pokemon_df.type2.value_counts()).plot.bar()

In [ ]:
## Note: you cannot simply add up all the frequencies since 
## not every Pokemon has 2 types, be careful with the data

In [ ]:
## Alternatively you can do this
fig = plt.figure(figsize=(15,15)) ## figsize=(width, height)
fig.add_subplot(211) # subplot(nrows, ncols, index)

## autopct: used to label the wedges with their numeric value; 
## the label will be placed inside the wedge
## pctdistance: the ratio between the center of each pie slice 
## and the start of the text generated by autopct
pokemon_df['type1'].value_counts().plot(kind = 'pie', 
                                       autopct = '%1.1f%%', 
                                       pctdistance = 0.9)

plt.show()

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.add_subplot(212)
pokemon_df['type2'].value_counts().plot(kind = 'pie', 
                                       autopct = '%1.1f%%',
                                       pctdistance = 0.9)

In [ ]:
## Other things we can look into, in no particular order
## The frequency of Pokemon (y) vs HP stat total:
pokemon_df.hp.value_counts().sort_index().plot.line()

In [ ]:
## Frequency with weight
pokemon_df.weight_kg.plot.hist()

### Some Pokemon doesn't have secondary type so they have NaN (null values) in the Type 2 column. Let's fill in the null values in the Type 2 column by replacing it with None

In [ ]:
## this is super important otherwise, later logistic regression 
## will throw an error could not convert string to float: 'None'
pokemon_df['type2'].fillna(value='None', inplace=True) 

## Pokemon type combinations
### We've already seen what is the most and least common type of Pokemon, it will be also interesting to see all the type combination of the Pokemon, note that we will not include Pokemon that doesn't have secondary type

In [ ]:
plt.subplots(figsize=(10, 10))

sns.heatmap(
    pokemon_df[pokemon_df['type2'] != 'None'].groupby(['type1', 'type2']).size().unstack(),
    linewidths = 1,
    annot = True,
    cmap = "Blues" # color
)

# plt.xticks(rotation = 35)
plt.show()

In [ ]:
# Produce a list of unique abilities across all Pokemon    
abilitiesList = []
for n in range(len(pokemon_df.abilities)):
    for ability in pokemon_df.abilities.iloc[n]:
        if ability not in abilitiesList:
            abilitiesList.append(ability)
            
print('Number of unique abilities: ', len(abilitiesList))

In [ ]:
## Suppose you want to know how effective certain types are to one another

data = {
    'attack': pokemon_df['attack'],
    'defense': pokemon_df['defense'],
    'sp_attack': pokemon_df['sp_attack'],
    'sp_defense': pokemon_df['sp_defense'],
    'type2': pokemon_df.type2,
    'type1': pokemon_df['type1']
}
data = pd.DataFrame(data)
## look for columns with 'against' and join them
data = pokemon_df.filter(like='against').join(data) 

X = data.drop('type1', axis=1)
y = data['type1']
print(list(X))
X

## Note you can can also generate a heatmap

# 2. Predicting a Pokemon's type from its abilities

In [ ]:
# Fix capture rate for Minior, then convert data to numeric
pokemon_df.capture_rate.iloc[773] = 255  
pokemon_df.capture_rate = pd.to_numeric(pokemon_df.capture_rate)

In [ ]:
# print('Number of samples = number of Pokemon = ', pokemon_df.shape[0])

# Processing Ability data
# Ability data is in a single string format: "['Ability1','Ability2']"
# Change it to a list of strings
def abilities_to_list(abil):
    abil_ = abil.replace(' ','')
    abil_ = abil_[2:-2].split("','")
    return abil_

# Cycle through ability data and transform it to list of string format
# Note: this step is slow
for n in range(len(pokemon_df.abilities)):
    pokemon_df.abilities.iloc[n] = abilities_to_list(pokemon_df.abilities.iloc[n])
            
# Create a new dataframe containing the relevant information            
dfAbilities = pokemon_df.loc[:,['pokedex_number','name','abilities','type1','type2']]
for n in range(len(dfAbilities.abilities)):
    dfAbilities.abilities.iloc[n] = " ".join(dfAbilities.abilities.iloc[n])

### We'll use 90% of our data to train the classifier, and the remaining 10% to test it. We need to randomize the order since Pokemon in the same evolution line usually have the same set of abilities.

In [ ]:
from pylab import ceil

# Define a function that splits our dataframe randomly into 
# train and test sets        
def train_test_split_manual(pokemon_df, split=0.9):
    n_samples = pokemon_df.shape[0]
    n_test = int(ceil(split*n_samples))
    indices = np.random.permutation(n_samples)
    X_train = pokemon_df.iloc[indices[:n_test]]
    X_test = pokemon_df.iloc[indices[n_test:]]
    return X_train, X_test
    
# Set the random seed so the results are the same each time we run this kernel
np.random.seed(801)
# Split the data
X_train, X_test = train_test_split_manual(dfAbilities)

In [ ]:
# Feature Extration: Build dictionary of features from Abilities
# CountVectorizer creates sparse matrices of 
# counts of each word in the dictionary
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train.abilities.values)          

# Apply tf-idf downscaling since some Pokemon have more abilities than others
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# Check the shape of the feature array
print(X_train_tfidf.shape)

In [ ]:
# Try to predict a pokemon's type1 and type2 from its abilities
# Use a Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB

# Fit the classifier to Type 1 training data:
classifier = MultinomialNB().fit(X_train_tfidf, X_train.type1.values)

# Transform the testing set to sparse feature matrices of counts
X_test_counts = count_vect.transform(X_test.abilities.values)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

predicted = classifier.predict(X_test_tfidf)
# Check the fraction of correct predictions
print("Fraction of correct predictions: Abilities predicting Type 1:")
print(predicted[predicted == X_test.type1.values].shape[0], '/', predicted.shape[0])
print((predicted[predicted == X_test.type1.values].shape[0])/float(predicted.shape[0]))

In [ ]:
# Try the prediction WITHOUT tf-idf downscaling
classifier = MultinomialNB().fit(X_train_counts, X_train.type1.values)
predicted = classifier.predict(X_test_counts)
# Check the fraction of correct predictions
print("Fraction of correct predictions: Abilities predicting Type 1 without tf-idf:")
print(predicted[predicted == X_test.type1.values].shape[0], '/', predicted.shape[0])
print((predicted[predicted == X_test.type1.values].shape[0])/float(predicted.shape[0]))

In [ ]:
# Fit the classifier to Type 2 training data:
# Preprocess type 2 data, since it contains null values when Pokemon
# have only one type and has mixed datatypes
type2 = np.array([str(value) for value in X_train.type2.values])
indices = type2 != 'nan'
classifier = MultinomialNB().fit(X_train_counts[indices], type2[indices])
type2_test = np.array([str(value) for value in X_test.type2.values])
indices_test = type2_test != 'nan'
predicted = classifier.predict(X_test_counts[indices_test])

# Check the fraction of correct predictions
print("Fraction of correct predictions: Abilities predicting Type 2 without tf-idf:")
print((predicted[predicted == type2_test[indices_test]].shape[0])/float(predicted.shape[0]))

### Conclusion: A Pokemon's 1st typing can be correctly predicted from its abilities alone abot 60% of the time, and its 2nd type 40% of the time.

# 3. Intro to Logistic Regression

### From Bulbapedia : Legendary Pokémon are a group of incredibly rare and often very powerful Pokémon, generally featured prominently in the legends and myths of the Pokémon world. 

### Can we predict whether or not a Pokemon is legendary?

In [ ]:
## Let's first see how many there are

fig = plt.figure(figsize=(7,7))

colors = ["aqua", "orange"]
pokeLeg = pokemon_df[pokemon_df['is_legendary'] == True]
pokeNon = pokemon_df[pokemon_df['is_legendary'] == False]

legDist = [pokeLeg['name'].count(), pokeNon['name'].count()]
legPie = plt.pie(legDist,
                 labels= ['Legendary', 'Non Legendary'], 
                 autopct ='%1.1f%%', 
                 shadow = True,
                 colors = colors,
                 startangle = 45,
                 explode=(0, 0.1))

In [ ]:
## correlation between legendary pokemon and non-legendary per generation
colors = ["aqua", "orange"]
g = sns.factorplot(
    x='generation', 
    data=pokemon_df,
    kind='count', 
    hue='is_legendary',
    palette=colors, 
    size=5, 
    aspect=1.5,
    legend=False,
    ).set_axis_labels('Generation', '# of Pokemon')

g.ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),  shadow=True, ncol=2, labels=['NON LEGENDARY','LEGENDARY'])
plt.show()

In [ ]:
## Not usually a good sign since there is not enough data on what is a legendary
## Maybe there's a very strong correlation we can exploit
## Method 1: High attack and defense correlation
## Method 2: hasGender - legendaries typically don't
## Method 3: Catch rate - legendaries have very low catch rate

In [ ]:
## I will show Method 1 because it goes back to reviewing linear regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()

X = pokemon_df[['attack']]
Y = pokemon_df[['defense']]
model.fit(X, Y)

Y_pred = model.predict(X)

plt.scatter(X, Y)
plt.plot(X, Y_pred, color = 'red')
plt.xlabel('Attack')
plt.ylabel('Defense')
plt.show()

In [ ]:
## there is another graph we can look at, called a hex graph, 
## which shows you where most of the points congregate
## The number of hexagons in the x-direction is gridsize
pokemon_df.plot.hexbin(x='defense', y='attack', gridsize=15) 

In [ ]:
## So, is there a relationship between attack and is_legendary?
sns.boxplot(x=pokemon_df.is_legendary, y=pokemon_df.attack)

In [ ]:
## What about defense and is_legendary?
sns.boxplot(x=pokemon_df.is_legendary, y=pokemon_df.defense)

In [ ]:
## KDE Plot described as Kernel Density Estimate is used for 
## visualizing the Probability Density of a continuous variable.
## Here is a representation of is_legendary with generations
grid = sns.FacetGrid(pokemon_df, col='is_legendary', row='generation')
grid.map(sns.kdeplot, 'attack')

In [ ]:
sns.boxplot(x='generation', y='base_total', hue='is_legendary', data=pokemon_df)

In [ ]:
sns.lmplot(x='attack', y='defense', hue='is_legendary', fit_reg=False, data=pokemon_df, markers = ['x', 'o'])

In [ ]:
## So it's not as strong as we would like it to be but it 
## turns out this is still a viable approach, let's incorporate it with
## gender and catch rate
leg = pokemon_df[pokemon_df['is_legendary'] == True]
pokemon_df['capture_rate'].set_value(773, '30')

## does not sort capture_rate properly not sure why, isn't 
## needed anyways, just for visualization
# plt.figure(figsize=(15,6))
# plt.title("Catch rate of Pokemon")
# pokemon_df['capture_rate'].sort_values()
# sns.scatterplot(x='pokedex_number', y='capture_rate', data=pokemon_df, hue='is_legendary')

In [ ]:
leg_total = leg['base_total'].mean()
non_leg_total = pokemon_df[pokemon_df['is_legendary'] != True]['base_total'].mean()

In [ ]:
pd.DataFrame([leg_total, non_leg_total], index=['Legendary', 'non-Legendary'], columns=['Average Total'])

In [ ]:
isLegendary = pd.get_dummies(pokemon_df['is_legendary'], drop_first=True)
lr_df = pokemon_df[['base_total', 'capture_rate']]
lr_df = pd.concat([lr_df, isLegendary], axis=1)

In [ ]:
lr_df.columns = ['Base Total', 'Catch Rate', 'Is Legendary']
lr_df.head()

In [ ]:
X = lr_df.drop('Is Legendary', axis = 1)
y = lr_df['Is Legendary']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)

In [ ]:
pred = LR.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y_test,pred))

# 4. Decision Trees Approach (Extra)

In [ ]:
from sklearn import tree
from sklearn.model_selection import cross_val_score, KFold
kfold = KFold(n_splits=10, random_state=48)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,y)

result = cross_val_score(clf, X, y, cv=kfold, scoring='accuracy')

print(result.mean())

# 4. Other Pokemon Related Problems / Resources

Trying to figure out what Pokemon you are (given an image of yourself, find a Pokemon that is most similar to you):
https://www.kaggle.com/kohlisaab/pokemon-comparison

Heat maps with various other measured data:
https://www.kaggle.com/jeru666/my-pokemon-collection

Beautiful Visualizations:
https://www.kaggle.com/xvivancos/analyzing-a-pok-mon-data-set-my-first-kernel

Generate new Pokemon images data set:
https://www.kaggle.com/russellgeum/dcgan-with-pokemon-high-resolution

Other approaches to is_legendary (neural network, xgboost, tpot approach?):
https://www.kaggle.com/yassinealouini/pokemons-machine-learning-101

Learn pandas with Pokemon:
https://www.kaggle.com/ash316/learn-pandas-with-pokemons

In depth Pokemon stats analysis:
https://www.kaggle.com/ndrewgele/visualizing-pok-mon-stats-with-seaborn

Even more models for predicting whether a Pokemon is legendary:
https://adityavichare.wordpress.com/2017/09/21/predicting-legendary-pokemon-using-dataset-of-stats/